In [10]:
import openai
OPENROUTER_API_KEY = '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'
YOUR_SITE_URL = 'www'
YOUR_APP_NAME = 'demo'
MODEL = 'mistralai/mixtral-8x7b-instruct'

# google/gemma-7b-it
# mistralai/mixtral-8x7b-instruct
# meta-llama/llama-2-70b-chat
# openai/gpt-3.5-turbo-instruct

# Define your constraints
max_tokens = 20  # Set your desired maximum word count here

# OpenRouter API base URL
openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = OPENROUTER_API_KEY

In [11]:
from os import getenv
from openai.error import APIError
import time

def use_model(model_name, content, max_retries=5):
    for i in range(max_retries):
        try:
            completion = openai.ChatCompletion.create(
            extra_headers={
                "HTTP-Referer": YOUR_SITE_URL, # Optional, for including your app on openrouter.ai rankings.
                "X-Title": YOUR_APP_NAME, # Optional. Shows in rankings on openrouter.ai.
            },
            model= model_name,
            messages=[
                {
                "role": "user",
                "content": content
                },
            ],
            max_tokens=max_tokens

            )
            return completion.choices[0].message.content
        except APIError as e:
            print(f"An error occurred: {e}. Retrying... (Attempt {i+1}/{max_retries})")
            time.sleep(2)  # Wait for 2 seconds before retrying
    print(f"Failed after {max_retries} attempts.")
    return None

In [12]:
# Parameters for generation
max_length = 5  # Maximum length of the generated output
import pandas as pd
import numpy as np

file_path_generated_description = 'Generated_text_mistralaimixtral-8x7b-instruct.csv'

# Read the CSV file into a DataFrame
df_text = pd.read_csv(file_path_generated_description)

df_text["Score"] = np.nan
df_text["Score_readability"] = np.nan
df_text["Score_relevance"] = np.nan
df_text["Score_descriptive language"] = np.nan
df_text["Score_grammatical correctness"] = np.nan

model_name_for_saving = MODEL.replace("/","")
model_name_for_saving = model_name_for_saving.replace(":","")

#for products with image
for index, row in df_text.iterrows():
    #check word count
    if len(row['Generated Product Description']) < 150: #fail word count test
        score = -2 
    else: #pass word count test
        scores = []
        #loop for each criteria
        for index, col in enumerate(['readability','relevance','descriptive language','grammatical correctness']):
            if index == 0:
                input_text = f"Please check if this product description is readable and please answer yes or no: {row['Generated Product Description']}"
            elif index == 1:
                input_text = f"Please check if this product description is relevant to the product name and please answer yes or no: Product Description: {row['Generated Product Description']} Product Name: {row['ProductName']}"
            elif index == 2:
                input_text = f"Please check if this product description uses descriptive languague and please answer yes or no: {row['Generated Product Description']}"
            else:
                input_text = f"Please check if this product description is grammatical correct and please answer yes or no: {row['Generated Product Description']}"

            #skip following checking if API fails
            if -1 in scores:
                continue
            output = use_model(MODEL, input_text)
        
            print(f"ItemSku: {row['ItemSku']}\nResult for {col}: {output}")
            
            if output == None:
                scores.append(-1)
            elif 'yes' in output.lower():
                scores.append(1)
            else:
                scores.append(0)
        
        if -1 in scores:
            score = -1
        else:
            score = sum(scores*25)
            df_text.loc[df_text['ItemSku'] == row['ItemSku'],["Score_readability"]] = scores[0]*25
            df_text.loc[df_text['ItemSku'] == row['ItemSku'],["Score_relevance"]] = scores[1]*25
            df_text.loc[df_text['ItemSku'] == row['ItemSku'],["Score_descriptive language"]] = scores[2]*25
            df_text.loc[df_text['ItemSku'] == row['ItemSku'],["Score_grammatical correctness"]] = scores[3]*25

    print(f"Score for {row['ItemSku']}: {score}")
    df_text.loc[df_text['ItemSku'] == row['ItemSku'],["Score"]] = score

    df_text.to_csv(f'Score_mistralaimixtral-8x7b_using-{model_name_for_saving}.csv',index=False)  #output{-2: fail word count, -1: fail API, others: score of description}





ItemSku: 19283315462
Result for readability: 
Yes, the product description is readable and informative. It provides detailed information about the candle
ItemSku: 19283315462
Result for relevance: 
Yes, the product description is relevant to the product name. It accurately describes the features, scent
ItemSku: 19283315462
Result for descriptive language: 
Yes, the product description uses descriptive language. It goes into detail about the fragr
ItemSku: 19283315462
Result for grammatical correctness: 
Yes, the product description is grammatically correct.
Score for 19283315462: 100
ItemSku: 88392962061
Result for readability: 
Yes, the product description is readable and provides a clear understanding of what the DVD includes,
ItemSku: 88392962061
Result for relevance: 
Yes, the product description is relevant to the product name. It describes the Game Night DVD,
ItemSku: 88392962061
Result for descriptive language: 
Yes, the product description uses descriptive language. It provides